In [1]:
pip install ultralytics opencv-python tensorflow


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ✅ Imports
import cv2
import numpy as np
import tensorflow as tf
from ultralytics import YOLO

# ✅ Paths to your trained models
yolo_model_path = "yolov8n.pt"  # Pre-trained YOLOv8 nano (small & fast)
classifier_model_path = r"C:\Users\sagni\Downloads\Trash-Detector\waste_classifier.h5"

# ✅ Load YOLOv8 model
yolo = YOLO(yolo_model_path)  # Use 'yolov8n.pt' or 'yolov8s.pt' for speed
print("✅ YOLOv8 model loaded.")

# ✅ Load your custom classifier
classifier = tf.keras.models.load_model(classifier_model_path)
class_labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
print("✅ Custom waste classifier loaded.")

# ✅ Preprocessing function for classifier
def preprocess_roi(roi):
    img = cv2.resize(roi, (224, 224))
    img = img.astype('float32') / 255.0
    return np.expand_dims(img, axis=0)

# ✅ Start webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Cannot access camera")
    exit()

print("📸 Camera started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to grab frame")
        break

    # Run YOLOv8 object detection
    results = yolo.predict(source=frame, save=False, imgsz=640, conf=0.5)

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes
        confs = result.boxes.conf.cpu().numpy()  # Confidence scores

        for box, conf in zip(boxes, confs):
            x1, y1, x2, y2 = [int(coord) for coord in box]

            # Extract ROI (Region of Interest) for classification
            roi = frame[y1:y2, x1:x2]
            if roi.size == 0: continue  # Skip empty regions

            # Classify the ROI with your custom model
            input_img = preprocess_roi(roi)
            preds = classifier.predict(input_img)
            class_idx = np.argmax(preds)
            confidence = np.max(preds)

            label = f"{class_labels[class_idx]} ({confidence*100:.2f}%)"

            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow("Smart Trash Detector (YOLO + Classifier)", frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
print("🛑 Camera closed.")


✅ YOLOv8 model loaded.


✅ Custom waste classifier loaded.
📸 Camera started. Press 'q' to quit.

0: 480x640 2 persons, 115.5ms
Speed: 7.1ms preprocess, 115.5ms inference, 8.4ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step

0: 480x640 2 persons, 64.8ms
Speed: 2.2ms preprocess, 64.8ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step

0: 480x640 2 persons, 60.4ms
Speed: 1.4ms preprocess, 60.4ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step

0: 480x640 2 persons, 61.1ms
Speed: 2.0ms preprocess, 61.1ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step

0: 480x640 2 persons, 62.1ms
Speed: 1.9ms preprocess, 62.1ms inference, 1.4ms postprocess per image at shape (1, 3, 